In [2]:
import cv2
import os
import yaml
from tqdm import tqdm

In [3]:
INPUT_BASE_DIR = 'C:/Empty_Parking/PKLot.v2-640.yolov8'
OUTPUT_BASE_DIR = 'C:/Empty_Parking/bounding_box'
DATA_YAML_PATH = os.path.join(INPUT_BASE_DIR, 'data.yaml')


In [4]:
def draw_boxes_for_split(split_name):
    """Processes all images in a given split (train, valid, or test)."""
    print(f"\nProcessing '{split_name}' split...")

    # Define paths for the current split
    image_dir = os.path.join(INPUT_BASE_DIR, split_name, 'images')
    label_dir = os.path.join(INPUT_BASE_DIR, split_name, 'labels')
    output_dir = os.path.join(OUTPUT_BASE_DIR, split_name)

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    if not os.path.exists(image_dir):
        print(f"Warning: Image directory not found for '{split_name}' split. Skipping.")
        return

    image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]

    for image_name in tqdm(image_files, desc=f"Drawing boxes on {split_name} images"):
        image_path = os.path.join(image_dir, image_name)
        label_path = os.path.join(label_dir, os.path.splitext(image_name)[0] + '.txt')

        # Read the image
        image = cv2.imread(image_path)
        if image is None:
            continue
        h, w, _ = image.shape

        # Check if the label file exists
        if not os.path.exists(label_path):
            # If no label, save a copy of the original image
            output_path = os.path.join(output_dir, image_name)
            cv2.imwrite(output_path, image)
            continue

        # Read labels and draw boxes
        with open(label_path, 'r') as f:
            for line in f.readlines():
                try:
                    class_id, x_center, y_center, width, height = map(float, line.strip().split())
                except ValueError:
                    continue # Skip malformed lines

                # Convert YOLO format to pixel coordinates
                box_w = int(width * w)
                box_h = int(height * h)
                center_x = int(x_center * w)
                center_y = int(y_center * h)
                x1 = center_x - box_w // 2
                y1 = center_y - box_h // 2
                x2 = x1 + box_w
                y2 = y1 + box_h

                # Get class name and color
                class_name = CLASS_NAMES.get(int(class_id), 'Unknown')
                color = CLASS_COLORS.get(class_name, (0, 0, 0)) # Default to black

                # Draw rectangle and label
                cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
                cv2.putText(image, class_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

        # Save the new image
        output_path = os.path.join(output_dir, image_name)
        cv2.imwrite(output_path, image)


In [5]:
if __name__ == '__main__':
    # Load class names from data.yaml
    try:
        with open(DATA_YAML_PATH, 'r') as f:
            data_yaml = yaml.safe_load(f)
            CLASS_NAMES = {i: name for i, name in enumerate(data_yaml['names'])}
    except Exception as e:
        print(f"Could not read {DATA_YAML_PATH}, using default names. Error: {e}")
        CLASS_NAMES = {0: 'space-empty', 1: 'space-occupied'}

    # Define colors for each class
    CLASS_COLORS = {
        'space-empty': (0, 255, 0),    # Green
        'space-occupied': (0, 0, 255) # Red
    }

    # Create the base output directory
    os.makedirs(OUTPUT_BASE_DIR, exist_ok=True)
    print(f"Output will be saved to: {OUTPUT_BASE_DIR}")

    # Process all splits
    splits_to_process = ['train', 'valid', 'test']
    for split in splits_to_process:
        draw_boxes_for_split(split)

    print("\nProcessing complete!")

Output will be saved to: C:/Empty_Parking/bounding_box

Processing 'train' split...


Drawing boxes on train images: 100%|████████████████████████████████████████████| 34197/34197 [05:06<00:00, 111.54it/s]



Processing 'valid' split...


Drawing boxes on valid images: 100%|██████████████████████████████████████████████| 2483/2483 [00:19<00:00, 129.22it/s]



Processing 'test' split...


Drawing boxes on test images: 100%|████████████████████████████████████████████████| 1242/1242 [00:14<00:00, 87.93it/s]


Processing complete!
